In [2]:
%load_ext autoreload
%autoreload 2

import keras
import math
import numpy as np
import tensorflow as tf

from utilsSimpleConv2D import*

from tensorflow.keras.layers import Dense,Layer
from typing import Tuple,List,Any,Dict
from tensorflow.python.keras import activations, initializers

In [112]:
class SpectralConv2D_T(Layer):

    def __init__(self, filters,
                 kernel_size=3,
                 strides=1,
                 padding='VALID',
                 use_lambda_out=False,
                 use_lambda_in=False,
                 use_encode=False,
                 use_decode=False,
                 trainable_omega_diag=True,
                 trainable_omega_triu=True,
                 trainable_omega_tril=True,
                 use_bias=False,
                 kernel_initializer="glorot_uniform",
                 activation="relu"):

        super(SpectralConv2D_T, self).__init__()

        self.filters = filters
        self.strides = strides
        self.padding = padding
        self.use_bias = use_bias
        self.kernel_size = kernel_size

        self.use_encode = use_encode
        self.use_decode = use_decode

        self.use_lambda_in = use_lambda_in
        self.use_lambda_out = use_lambda_out

        self.trainable_omega_tril=trainable_omega_tril
        self.trainable_omega_triu=trainable_omega_triu
        self.trainable_omega_diag=trainable_omega_diag

        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        input_shape = tf.TensorShape(input_shape)
        self.size_omega_part=self.filters*self.kernel_size*math.floor((self.kernel_size - 1) / 2)
        self.pad = math.floor((self.kernel_size - 1) / 2)

        # -----------------------------------------matrix_pad------------------------------------
        if self.padding == "SAME":
            if self.strides > 1:
                raise Exception("Not implemented: paddind=SAME and strides>1. if padding=SAME, strides=1")

            # Right_shape
            self.Right_shape: Tuple = input_shape[1] + 2 * self.pad, input_shape[2] + 2 * self.pad
            inputShape = input_shape[1], input_shape[2]
            # matrix_pad
            self.matrix_pad = build_matrix_padding(input_shape=inputShape, pad=self.pad)

        elif self.padding == "VALID":
            # Right_shape
            self.Right_shape: Tuple = input_shape[1], input_shape[2]
            # matrix_pad
            self.matrix_pad = tf.constant(np.identity(input_shape[1] * input_shape[2]), dtype="float32")

        else:
            raise Exception("Padding not found")

        # -------------------------------------------------------------------------------------

        # ------------------------------out_in_shape_phi_indices-------------------------------
        self.set_indices_phi()
        self.set_indices_ul_triangular()
        # -------------------------------------------------------------------------------------

        # \omega_diag
        if self.trainable_omega_diag:
            self.omega_diag = self.add_weight(
                name='omega_diag',
                shape=(self.filters, self.kernel_size,),
                initializer=self.kernel_initializer,
                dtype=tf.float32,
                trainable=self.trainable_omega_diag)
        
        else:
            self.omega_diag =tf.constant(np.random.uniform(size=(self.filters, self.kernel_size,),low=-0.05, high=0.05),dtype=tf.float32,name='omega_diag')

        # \omega_triu
        if self.trainable_omega_triu:
            self.omega_triu = self.add_weight( 
                name='omega_triu',
                shape=(1,self.size_omega_part),
                initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05, seed=None),
                dtype=tf.float32,
                trainable=self.trainable_omega_triu)
        
        else:
            self.omega_triu =tf.constant(np.random.uniform(size=(1,self.size_omega_part),low=-0.05, high=0.05),dtype=tf.float32,name='omega_triu')

        # \omega_tril
        if self.trainable_omega_tril:
            self.omega_tril = self.add_weight( 
                name='omega_tril',
                shape=(1,self.size_omega_part),
                initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05, seed=None),
                dtype=tf.float32,
                trainable=self.trainable_omega_tril)
        
        else:
            self.omega_tril =tf.constant(np.random.uniform(size=(1,self.size_omega_part),low=-0.05, high=0.05),dtype=tf.float32,name='omega_tril')

        #---------------------------------indeice_Omega:part-----------------------------------
        self.set_indices_ul_triangular()
        # -------------------------------------------------------------------------------------


        
        # \use_lambda_in
        if self.use_lambda_in:
            self.use_lambda_in =self.add_weight(name='use_lambda_in',
                                        shape=(self.kernel_size,1),
                                        initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05, seed=None),
                                        dtype=tf.float32,
                                        trainable=self.use_lambda_in)

        else:
            self.use_lambda_in =tf.random.uniform(shape=(self.kernel_size,1),minval=-0.05,maxval=0.05,dtype=tf.float32,name='use_lambda_in')

        # \use_lambda_out
        if self.use_lambda_out:
            self.use_lambda_out =self.add_weight(name='use_lambda_out',
                                        shape=(1,self.kernel_size),
                                        initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05, seed=None),
                                        dtype=tf.float32,
                                        trainable=self.use_lambda_out)
        else:
            self.use_lambda_out = tf.random.uniform(shape=(1,self.kernel_size),minval=-0.05,maxval=0.05,dtype=tf.float32,name='use_lambda_out')



        # use_encode
        if self.use_encode:
            self.use_encode =self.add_weight(name='use_encode',
                                        shape=(1, self.Right_shape[0] * self.Right_shape[1]),
                                        initializer=tf.ones_initializer(),
                                        dtype=tf.float32,
                                        trainable=self.use_encode)
        else:
            
            self.use_encode = tf.ones(shape=(1, self.Right_shape[0] * self.Right_shape[1]),dtype=tf.float32, name='use_encode')


        # use_decode
        if self.use_decode:
            self.use_decode=self.add_weight(name='use_decode',
                                            shape=(self.output_lenght, 1),
                                            initializer=tf.zeros_initializer(),
                                            dtype=tf.float32,
                                            trainable=self.use_decode)
        
        else:
            self.use_decode = tf.zeros(shape=(self.output_lenght, 1),dtype=tf.float32, name='use_decode')

        
        # --------------------------------------------bias---------------------------------------
        if self.use_bias:
            self.bias = self.add_weight(
                name='bias',
                shape=(self.filters,),
                dtype=tf.float32,
                trainable=self.use_bias)
        else:
            self.bias = None
        # ---------------------------------------------------------------------------------------
        
        # ---------------------------------------------------------------------------------------
        self.build = True
        # ---------------------------------------------------------------------------------------
                                


    def call(self, inputs):

        #----------------------------------Inputs---------------------------------------------
        flatten = tf.reshape(inputs, shape=(-1, inputs.shape[1] * inputs.shape[2], inputs.shape[3]))
        upFlatten =tf.matmul(a=self.matrix_pad, b=flatten)
        upFlatten=tf.math.reduce_mean(upFlatten,axis=-1)


        #----------------------------------\Omega---------------------------------------------
        omega_high =tf.sparse.SparseTensor(
            indices=self.indices_triu, values=self.omega_triu[0],
            dense_shape=(self.filters, self.kernel_size, self.kernel_size)
        )
        omega_lower =tf.sparse.SparseTensor(
            indices=self.indices_tril, values=self.omega_tril[0],
            dense_shape=(self.filters, self.kernel_size, self.kernel_size)
        )
        omega_high = tf.sparse.to_dense(omega_high)
        omega_lower = tf.sparse.to_dense(omega_lower)
        omega=omega_lower+tf.linalg.diag(self.omega_diag, k=0)+omega_high

        #----------------------------------Base-----------------------------------------------
        
        base = tf.multiply(omega,self.use_lambda_in - self.use_lambda_out) 
        
        # ------------------------------Build Noyau--------------------------------------------

        kernel = tf.reshape(base,shape=(self.filters, self.kernel_size*self.kernel_size))

        kernel = tf.repeat(kernel, repeats=self.output_lenght, axis=0, name=None)

        kernel = tf.reshape(kernel, shape=(-1, self.filters * self.output_lenght * self.kernel_size * self.kernel_size))

        kernel = tf.sparse.SparseTensor(
            indices=self.indices, values=kernel[0],
            dense_shape=(self.filters, self.output_lenght, self.Right_shape[0] * self.Right_shape[1])
        )

        kernel = tf.sparse.to_dense(kernel)

        kernel = tf.linalg.matmul(kernel, tf.linalg.diag(self.use_encode[0, :], k=0)) - tf.linalg.matmul(
            tf.linalg.diag(self.use_decode[:, 0], k=0), kernel)
        
        #-----------------------------------Outputs-------------------------------------------------
        outputs = tf.transpose(tf.matmul(a=kernel, b=upFlatten,transpose_b=True))

        outputs = tf.reshape(outputs, shape=(-1, self.out_shape1, self.out_shape2,self.filters))

        if self.use_bias:
            outputs = tf.nn.bias_add(outputs, self.bias)

        if self.activation is not None:
            outputs = self.activation(outputs)
        else:
            pass

        return outputs
    
    def set_indices_phi(self, *args):
        self.indices: List[Tuple] = list()

        self.out_shape1: int = math.floor((self.Right_shape[0] - self.kernel_size) / self.strides) + 1
        self.out_shape2: int = math.floor((self.Right_shape[1] - self.kernel_size) / self.strides) + 1
        self.output_lenght: int = self.out_shape1 * self.out_shape2

        for filters in range(self.filters):
            count: int = 1
            shift: int = 0
            for i in range(self.output_lenght):
                if i == count * (self.out_shape2):
                    count += 1
                    shift += self.kernel_size + (self.strides - 1) * self.Right_shape[1]
                else:
                    if shift:
                        shift += self.strides
                    else:
                        shift += 1
                for block in range(self.kernel_size):
                    for j in range(self.kernel_size):
                        self.indices.append((filters, i, block * self.Right_shape[1] + shift - 1 + j))
    
    def set_indices_ul_triangular(self, *args):
        self.indices_triu: List[Tuple] = list()
        self.indices_tril: List[Tuple] = list()

        for filters in range(self.filters):
            for i in range(1,self.kernel_size):
                for j in range(i):
                    self.indices_tril.append((filters, i,j))

        for filters in range(self.filters):
            for i in range(self.kernel_size):
                for j in range(i+1,self.kernel_size):
                    self.indices_triu.append((filters, i,j))


        

    def get_indices_phi(self, *args):
        return self.indices

    def get_base(self, *args):
        #----------------------------------\Omega---------------------------------------------
        omega_high =tf.sparse.SparseTensor(
            indices=self.indices_triu, values=self.omega_triu[0],
            dense_shape=(self.filters, self.kernel_size, self.kernel_size)
        )
        omega_lower =tf.sparse.SparseTensor(
            indices=self.indices_tril, values=self.omega_tril[0],
            dense_shape=(self.filters, self.kernel_size, self.kernel_size)
        )
        omega_high = tf.sparse.to_dense(omega_high)
        omega_lower = tf.sparse.to_dense(omega_lower)
        omega=omega_lower+tf.linalg.diag(self.omega_diag, k=0)+omega_high
        #----------------------------------Base-----------------------------------------------
        
        base = tf.multiply(omega,self.use_lambda_in - self.use_lambda_out) 
        return base


In [1]:
class SpecConv2D(Layer):

    def __init__(self, filters,
                 kernel_size=3,
                 use_lambda_in=True,
                 use_bias=False,
                 activation="relu"):

        super(SpecConv2D, self).__init__()

        self.filters = filters
        self.use_bias = use_bias
        self.kernel_size = kernel_size
        self.use_lambda_in = use_lambda_in
        self.activation = activations.get(activation)

        
        self.initializer = initializers.RandomUniform(-1, 1)

    def build(self, input_shape):
        input_shape = tf.TensorShape(input_shape)
        self.input_channel = input_shape[-1]
        # ------------------------------out_in_shape_phi_indices---------------------------
        self.set_indices_phi(N=input_shape[1],M=input_shape[2])
        # -----------------------------------------------------------------------------------
    
        # --------------------------------noyau_of_phi----------------------------------------
        self.noyau_of_phi = tf.constant(np.ones((self.filters,
                                                self.kernel_size * self.kernel_size)),
                                                dtype="float32")
        
        # -----------------------------------kernel-------------------------------------------
        kernel = tf.repeat(self.noyau_of_phi, repeats=self.output_lenght, axis=0, name=None)
        
        kernel = tf.reshape(kernel, shape=(-1, self.filters * self.output_lenght * self.kernel_size * self.kernel_size))
        
        kernel = tf.sparse.SparseTensor(
        indices=self.indices, values=kernel[0],
        dense_shape=(self.filters, self.output_lenght, input_shape[1]*input_shape[2])
        )
        
        self.kernel = tf.sparse.to_dense(kernel)
        # -------------------------------------------------------------------------------------

        # Lambda_in
        if self.use_lambda_in:
            self.Lambda_in = self.add_weight(
                name='Lambda_in',
                shape=(1, input_shape[1]*input_shape[2]),
                initializer=self.initializer,
                dtype=tf.float32,
                trainable=self.use_lambda_in)

        else:
            raise Exception("Not implemented.")

        # --------------------------------------------bias---------------------------------------
        if self.use_bias:
            self.bias = self.add_weight(
                name='bias',
                shape=(self.filters,),
                dtype=tf.float32,
                trainable=self.use_bias)
        else:
            self.bias = None
        # ---------------------------------------------------------------------------------------

        # ---------------------------------------------------------------------------------------
        self.build = True
        # ---------------------------------------------------------------------------------------
    
    def set_indices_phi(self,N:int,M:int, *args):
        self.indices: List[Tuple] = list()
    
        self.out_shape1: int = math.floor((N - self.kernel_size) / self.kernel_size) + 1
        self.out_shape2: int = math.floor((M - self.kernel_size) / self.kernel_size) + 1
        self.output_lenght: int = self.out_shape1 * self.out_shape2
    
        for filters in range(self.filters):
            count: int = 1
            shift: int = 0
            for i in range(self.output_lenght):
                if i == count * (self.out_shape2):
                    count += 1
                    shift += self.kernel_size + (self.kernel_size - 1) * M
                else:
                    if shift:
                        shift += self.kernel_size
                    else:
                        shift += 1
                for block in range(self.kernel_size):
                    for j in range(self.kernel_size):
                        self.indices.append((filters, i, block * M + shift - 1 + j))

    def get_indices_phi(self, *args):
        return self.indices

    def call(self, inputs):

        
        # -----------------------------------------------------------------------------------------------------
        flatten = tf.reshape(inputs, shape=(-1, inputs.shape[1] * inputs.shape[2], inputs.shape[3]))
        # -----------------------------------------------------------------------------------------------------
        upFlatten=tf.math.reduce_mean(flatten,axis=-1)
        # -----------------------------------------------------------------------------------------------------
        outputs = tf.transpose(tf.matmul(a=tf.linalg.matmul(self.kernel, tf.linalg.diag(self.Lambda_in[0, :], k=0)), b=upFlatten,transpose_b=True))

        outputs = tf.reshape(outputs, shape=(-1, self.out_shape1, self.out_shape2,self.filters))
        # -----------------------------------------------------------------------------------------------------#

        if self.use_bias:
            outputs = tf.nn.bias_add(outputs, self.bias)

        if self.activation is not None:
            outputs = self.activation(outputs)
        else:
            pass

        return outputs


    def get_kernel(self, *args):
        return self.kernel

NameError: name 'Layer' is not defined

In [96]:
## premiere approche
N=6
M=6
F=3
S=1
filters=1
parameters={ "use_lambda_out":False,
                 "use_lambda_in":False,
                "use_encode":False,
                 "use_decode":False,
                 "trainable_omega_diag":True,
                 "trainable_omega_triu":True,
                 "trainable_omega_tril":True,
                 "kernel_initializer":"glorot_uniform"
           }




In [114]:
layer=(SpectralConv2D_T(filters=2, kernel_size=3, strides=1, padding="SAME", activation=None,**parameters))
#layer(tf.keras.layers.Input(shape=(28 , 28,1), dtype='float32'))

In [106]:
img=np.random.normal(0, 1,size=(1,N,M,1))
print(f"Image d'entrée: {img}")

Image d'entrée: [[[[ 0.88049796]
   [ 0.35491201]
   [ 0.11263292]
   [ 0.72871133]
   [ 1.1616679 ]
   [-0.38497808]]

  [[ 0.18875233]
   [-0.07296122]
   [-0.33306842]
   [-0.7492571 ]
   [ 0.23491477]
   [ 1.71182306]]

  [[ 0.36004349]
   [-1.23203906]
   [-0.53227091]
   [ 0.66694631]
   [ 0.43953803]
   [-0.41674727]]

  [[ 1.06018379]
   [ 0.86103843]
   [ 0.09868164]
   [ 1.05446657]
   [ 0.49621513]
   [ 0.74699034]]

  [[-1.04040859]
   [ 0.24225563]
   [ 2.51151412]
   [-0.56204424]
   [ 0.69344789]
   [ 1.51736147]]

  [[ 2.77989466]
   [-0.46763324]
   [ 2.77353635]
   [-1.25816263]
   [ 1.91171188]
   [-0.0421498 ]]]]


In [115]:
outs=layer(img)
outs.shape

TensorShape([1, 6, 6, 2])

In [116]:
print(f"Convolution Matricielle:\n {outs[0,:,:,0]}")

Convolution Matricielle:
 [[-0.03388312 -0.01066039  0.00350032 -0.03516895 -0.07000513  0.01377967]
 [ 0.00999247  0.01290107  0.00461747  0.02250127 -0.00352005 -0.06335004]
 [-0.0226194   0.04914146  0.00745316 -0.03318024 -0.02808965  0.01660553]
 [-0.04679346 -0.07050359 -0.00193469 -0.05031361 -0.04282536 -0.02726573]
 [ 0.0468843  -0.0464943  -0.07646619 -0.00765976 -0.02751264 -0.05641139]
 [-0.10639509  0.00381083 -0.10324342  0.04556541 -0.07340789 -0.00011456]]


In [117]:
print(f"Convolution Matricielle:\n {outs[0,:,:,1]}")

Convolution Matricielle:
 [[ 0.04649356  0.0090134  -0.0150525   0.05140681  0.11784066 -0.02223207]
 [-0.02894736 -0.02474784  0.00263202 -0.0260685  -0.00036793  0.08847621]
 [ 0.04277377 -0.06886703  0.00394329  0.05334235  0.04664207 -0.02504629]
 [ 0.0682451   0.12740043 -0.0069195   0.07734222  0.07546207  0.03681482]
 [-0.07247841  0.0972991   0.09204569  0.02640494  0.03816412  0.07686839]
 [ 0.1502364  -0.01353892  0.14660646 -0.07160401  0.10572207 -0.00432332]]


In [118]:
omega=layer.get_base()

In [119]:
print(f"Convolution Classique:\n {convolution(img=img[0,:,:,0],kernel=omega[0])}")

Convolution Classique:
 [[-0.03388312 -0.01066039  0.00350032 -0.03516895 -0.07000513  0.01377967]
 [ 0.00999247  0.01290107  0.00461748  0.02250127 -0.00352005 -0.06335005]
 [-0.0226194   0.04914146  0.00745316 -0.03318024 -0.02808965  0.01660553]
 [-0.04679346 -0.07050359 -0.00193469 -0.05031361 -0.04282536 -0.02726573]
 [ 0.0468843  -0.0464943  -0.0764662  -0.00765976 -0.02751264 -0.05641139]
 [-0.1063951   0.00381083 -0.10324341  0.04556542 -0.07340789 -0.00011456]]


In [120]:
print(f"Convolution Classique:\n {convolution(img=img[0,:,:,0],kernel=omega[1])}")

Convolution Classique:
 [[ 0.04649356  0.0090134  -0.0150525   0.05140682  0.11784067 -0.02223207]
 [-0.02894736 -0.02474784  0.00263201 -0.0260685  -0.00036793  0.0884762 ]
 [ 0.04277377 -0.06886702  0.0039433   0.05334235  0.04664207 -0.02504629]
 [ 0.06824509  0.12740043 -0.0069195   0.07734222  0.07546207  0.03681482]
 [-0.07247841  0.0972991   0.09204569  0.02640494  0.03816412  0.07686838]
 [ 0.1502364  -0.01353892  0.14660645 -0.07160401  0.10572206 -0.00432332]]


In [47]:
## Seconde Approche

In [59]:
N=6
M=6
F=3
S=3
filters=1
index=indices_phi(filters=filters,N=N,M=M,F=F,S=S)

In [60]:
out_shape1:int=math.floor((N-F)/S)+1
out_shape2:int=math.floor((M-F)/S)+1
output_lenght:int=out_shape1*out_shape2
noyau=tf.constant(np.ones((filters, F*F)),dtype="float32")
print(f"noyau: {noyau}")

noyau: [[1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [61]:
kernel=tf.repeat(noyau, repeats=output_lenght, axis=0, name=None)
kernel=tf.reshape(kernel,shape=(-1,filters*output_lenght*F*F))
kernel=tf.sparse.SparseTensor(
indices=index, values=kernel[0], dense_shape=(filters,output_lenght,N*M)
)
kernel=tf.sparse.to_dense(kernel)
print(f"Matrice de convolution M: {kernel}")

Matrice de convolution M: [[[1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
   0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
   1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1.]]]


In [62]:
img=np.random.normal(0, 1,size=(N,M))
print(f"Image d'entrée: {img}")

Image d'entrée: [[-0.22246216 -2.09627529 -0.36918137  0.1195452   0.32177229  0.65963848]
 [-0.46130394 -0.58163001 -0.11297976 -1.43243222  0.14071547 -1.50724429]
 [ 0.03488263  0.15866136  0.03607143 -0.99991941 -1.59708492  0.0809257 ]
 [-1.14374806  0.16731095 -0.47600588  0.63772628  0.00901906  1.50442779]
 [ 0.13341463  0.14854315 -0.40912516 -0.3888402   1.14209035  0.74235159]
 [-1.22165522 -1.83637194  0.10667412  0.35587417 -0.45208858  1.17346321]]


In [63]:
noyau_conv=np.ones((F,F),dtype="float32")

In [64]:
print(f"Convolution Classique:\n {convolution(img=img,kernel=noyau_conv)}")

Convolution Classique:
 [[-3.3616714  -3.8438325  -4.4729533  -1.3325604  -1.6980051  -0.38511807]
 [-3.1681273  -3.614217   -5.27814    -3.8934934  -4.2140837  -1.9012773 ]
 [-1.8258271  -2.3787413  -2.6031973  -3.79489    -3.1638665  -1.3692412 ]
 [-0.5009354  -1.349995   -1.1255774  -2.0460684   1.1306963   1.8817296 ]
 [-3.7525065  -4.5309634  -1.6942145   0.52532417  4.724024    4.1192636 ]
 [-2.7760694  -3.0785205  -2.0232458   0.35458472  2.5728505   2.6058166 ]]


In [65]:
inputs=tf.constant(img,dtype="float32")
inputs=tf.reshape(inputs, shape=(N*M,1))
print(f"Convolution Matricielle:\n {tf.reshape(tf.matmul(a=kernel,b=inputs),shape=(out_shape1,out_shape2))}")

Convolution Matricielle:
 [[-3.6142173 -4.2140837]
 [-4.5309634  4.724024 ]]
